Exponential Smoothing. Many forecasting models use parameters that are estimated
using nonlinear optimization. A good example is the Bass model introduced in this chapter.
Another example is the exponential smoothing forecasting model which is a common fore-
casting model used in practice. For instance, the basic exponential smoothing model for fore-
casting sales is

$\hat{y}_{t+1} = ay_t + (1 - a) \hat{y}_t$

where

$\hat{y}_{t+1}$ = forecast of sales for period t + 1

$y_t$ = actual sales for period t

$\hat{y}_t$ = forecast of sales for period t

$\alpha{}$ = smoothing constant, 0 <= a <= 1

This model is used recursively; the forecast for time period t + 1 is based on the
forecast for period t, $ŷ_t$ , the observed value of sales in period t, $y_t$ , and the smoothing
parameter a . The use of this model to forecast sales for 12 months is illustrated in the
following table with the smoothing constant $\alpha$ = 0.3. The forecast errors, $y_t - \hat{y}_t$ , are
calculated in the fourth column. The value of a is often chosen by minimizing the sum
of squared forecast errors. The last column of the table shows the square of the forecast
error and the sum of squared forecast errors.

The file ExpSmooth contains the observed data shown here. Construct this table
using the formula above. Note that we set the forecast in period 1 to the observed
in period 1 to get started ( $\hat{y}_1$ = $y_1$ = 17 ), then the formula above for $\hat{y}_{t+1}$  is used
starting in period 2. Make sure to have a single cell corresponding to a in your
spreadsheet model. After confirming the values in the table below with $\alpha$ = 0.3, try
different values of a to see if you can get a smaller sum of squared forecast errors.

In [2]:
# Week  |   Observed Value |  Forecast Value |   Error      |   Squared Error
# (t)   |  (y_t)           |  (y^_t)         |   (yt - y^t) |   (yt - y^t)^2
# 1     |  17              |  17.00          |   0.00       |   0.00
# 2     |  21              |  17.00          |   4.00       |   16.00
# 3     |  19              |  18.20          |   0.80       |   0.64
# 4     |  23              |  18.44          |   4.56       |   20.79
# 5     |  18              |  19.81          |  -1.8        |   13.27
# 6     |  16              |  19.27          |  -3.2        |   710.66
# 7     |  20              |  18.29          |   1.71       |   2.94
# 8     |  18              |  18.80          |  -0.80       |   0.64
# 9     |  22              |  18.56          |   3.44       |   11.83
# 10    |  20              |  19.59          |   0.41       |   0.17
# 11    |  15              |  19.71          |  -4.71       |   22.23
# 12    |  22              |  18.30          |   3.70       |   13.69
# SUM   |  --              |  --             |   --         |   102.86

data = {'week': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        'true': [17, 21, 19, 23, 18, 16, 20, 18, 22, 20, 15, 22],
        'predicted': [17.00, 17.00, 18.20, 18.44, 19.81, 19.27, 18.29, 18.80, 18.56, 19.59, 19.71, 18.30],
        'error': [0.00, 4.00, 0.80, 4.56, -1.8, -3.2, 1.71, -0.80, 3.44, 0.41, -4.71, 3.70],
        'error_squared': [0.00, 16.00, 0.64, 20.79, 13.27, 710.66, 2.94, 0.64, 11.83, 0.17, 22.23, 13.69]}


In [41]:
from docplex.cp.model import CpoModel as Model


m = Model()

alpha = m.integer_var(name = 'alpha')

m.add_constraint(alpha >= 0)
m.add_constraint(alpha <=1000)

alpha_float = alpha/1000

start_value = 17

predicted = [17]

for i in range(1, len(data['week'])):
    predicted.append(alpha_float * data['true'][i-1] + (1 - alpha_float) * predicted[i-1])

error = [(pred - tru)**2 for pred, tru in zip(predicted, data['true'])]

m.minimize(m.sum(error))
solution = m.solve(log_output = True)

 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Minimization problem - 1 variable, 2 constraints
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 10.0 (before), 10.0 (after)
 !  . Memory usage      : 299.7 kB (before), 299.7 kB (after)
 ! Using parallel search with 4 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0          1                 -
 + New bound is 16
                        0          1    1            -
 + New bound is 35.16110
 *           131        1  0.08s        1      (gap is 73.16%)
 *      130.5318        3  0.08s        1      (gap is 73.06%)
 *      114.6853        4  0.08s        1      (gap is 69.34%)
 *      106.5805       13  0.08s        1      (gap is 67.01%)
 *      106.4031      151  0.08s        1      (gap is 66.95%)
 *      105.3095        2  

 *      98.74611       16  0.08s        2      (gap is 64.39%)
 *      98.59555      765  0.08s        3      (gap is 64.34%)
        98.59555     1000          1    3   F    11  = alpha
 *      98.56407       18  0.08s        4      (gap is 64.33%)
        98.56407     2000          1    3   F     7  = alpha
        98.56407     3000          1    3   F        -
        98.56407     1000          1    1   F   836  = alpha
 *      98.55956      698  0.18s        2      (gap is 64.33%)
        98.55956     4000          1    3   F   659  = alpha
        98.55956     1000          1    2       850 != alpha
        98.55956     5000          1    3   F   193  = alpha
 ! Time = 0.25s, Average fail depth = 8, Memory usage = 3.2 MB
 ! Current bound is 35.16110 (gap is 64.33%)
 !          Best Branches  Non-fixed    W       Branch decision
        98.55956     1000          1    4       300 != alpha
        98.55956     6000          1    3   F   320  = alpha
        98.55956     7000        

In [31]:
m.export_as_cpo('alpha.cpo')

In [42]:
print(solution[alpha]/1000," alpha")
print(solution.get_objective_value())

0.173  alpha
98.55956125237986


In [40]:
other_pred =  [17]

thing_alpha = 0.18

for i in range(1, len(data['week'])):
    other_pred.append(thing_alpha * data['true'][i-1] + (1 - thing_alpha) * other_pred[i-1])

error = [(pred - tru)**2 for pred, tru in zip(other_pred, data['true'])]
sum(error)

98.57150185864782